In [ ]:
import os
import pandas as pd
import pprint

from pathlib import Path
from transformers import pipeline

from src import sentiment, helper, utility, models
from src.config import MBertConfig, SharedConfig
from src.metrics import evaluate_pipe

os.environ["TOKENIZERS_PARALLELISM"] = "false"

if __name__ == "__main__":
    # helper.jsonToCSV("miko.json", "miko.csv")
    
    jsonl = helper.read_jsonl_as_string(Path("miko.jsonl"))
    train_ds, val_ds, test_ds, label2id, id2label = utility.load_split_dataset(jsonl)  # load again, un-mapped

    texts = helper.to_list_str(test_ds[SharedConfig.TEXT_COL])
    labels = list(test_ds[SharedConfig.LABEL_COL])
    num_labels = len(id2label)

    context = models.setup_pipeline(MBertConfig)
    
    mBertTrainer = sentiment.train(context)
    mBertPipe = pipeline(
        "text-classification", 
        model=mBertTrainer.model, 
        tokenizer=mBertTrainer.tokenizer, 
        return_all_scores=True,
        device=0 if SharedConfig.USE_FP16 else -1
    )
    metrics = evaluate_pipe(mBertPipe, texts, labels, id2label=mBertTrainer.model.config.id2label)
    print("\r\nMetrics:")
    pprint.pprint(metrics)
    
    #sample_texts = [
    #    "Maganda ang serbisyo at mabilis ang delivery!",  # Tagalog positive
    #    "Sobrang pangit ng karanasan ko.",                # Tagalog negative
    #    "It was okay, nothing special.",                  # English neutral-ish
    #]
    #sentiment.infer(sample_texts, trainer.tokenizer, trainer.model)


/home/miko/miniconda/envs/mv_nlp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/miko/miniconda/envs/mv_nlp/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Map: 100%|█████████████████████████████████████████████████████████████████████████████| 21/21 [00:00<00:00, 8935.82 examples/s]
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']

Epoch,Training Loss,Validation Loss
